# **13.6 Custom_Exceptions**

Built-in exceptions like `ValueError` and `TypeError` are general purpose. When your Pokemon game grows large, you want errors that are specific to your domain — `InvalidTeamError`, `PokemonFaintedError`, `PokedexNotFoundError`. Custom exceptions make your code more readable, let callers catch *your* errors precisely without accidentally catching unrelated ones, and allow you to attach extra context like the Pokemon's name or battle turn number.

---

## **Creating a Basic Custom Exception**

All you need is a class that inherits from `Exception` (or any of its subclasses). Even an empty class body gives you a new, uniquely catchable exception type with its own name.

In [ ]:
# A minimal custom exception — just inherit from Exception
class PokemonError(Exception):
    """Base class for all Pokemon game errors."""
    pass

class InvalidLevelError(Exception):
    """Raised when a Pokemon level is outside the valid 1–100 range."""
    pass

class TeamFullError(Exception):
    """Raised when trying to add a 7th Pokemon to the active team."""
    pass

# Raise and catch them exactly like built-in exceptions
def set_level(level: int) -> int:
    if not (1 <= level <= 100):
        raise InvalidLevelError(f"Level {level} is out of range (must be 1–100)")
    return level

try:
    set_level(150)
except InvalidLevelError as e:
    print(f"Caught InvalidLevelError: {e}")

# Built-in except won't catch it — it's a separate type
try:
    set_level(0)
except ValueError:
    print("This won't print")
except InvalidLevelError as e:
    print(f"Correctly caught: {e}")

---

## **Adding Extra Information**

Custom exceptions can store extra data alongside the message. Override `__init__` to accept additional parameters, then callers can inspect them without parsing the message string.

In [ ]:
class InvalidLevelError(Exception):
    """
    Raised when a Pokemon level is invalid.
    Stores the pokemon name and the bad level as attributes.
    """
    def __init__(self, pokemon_name: str, level: int, min_level=1, max_level=100):
        self.pokemon_name = pokemon_name
        self.level        = level
        self.min_level    = min_level
        self.max_level    = max_level
        # Build a descriptive message and pass it to the parent
        super().__init__(
            f"{pokemon_name}'s level {level} is outside "
            f"the valid range {min_level}–{max_level}"
        )

def create_pokemon(name: str, level: int) -> dict:
    if not (1 <= level <= 100):
        raise InvalidLevelError(name, level)  # Rich exception with context
    return {"name": name, "level": level}

try:
    create_pokemon("Mewtwo", 150)
except InvalidLevelError as e:
    print(f"Error message : {e}")
    print(f"Pokemon       : {e.pokemon_name}")
    print(f"Bad level     : {e.level}")
    print(f"Valid range   : {e.min_level}–{e.max_level}")

---

## **Exception Hierarchies for Your Domain**

Just as Python's built-ins form a tree (`LookupError` → `KeyError`, `IndexError`), you can build your own hierarchy. A base `PokemonError` lets callers catch all game errors with one clause, while sub-classes let them catch only specific ones.

In [ ]:
# Base exception for the entire game
class PokemonError(Exception):
    """Root of all Pokemon game exceptions."""

# Second tier — error categories
class BattleError(PokemonError):
    """All battle-related errors."""

class TeamError(PokemonError):
    """All team management errors."""

class PokedexError(PokemonError):
    """All Pokedex lookup errors."""

# Third tier — specific errors
class PokemonFaintedError(BattleError):
    """Raised when trying to use a fainted Pokemon in battle."""

class TeamFullError(TeamError):
    """Raised when adding to a full 6-member team."""

class PokemonNotFoundError(PokedexError):
    """Raised when a Pokemon is not in the Pokedex."""

# Demonstrate the hierarchy
def attempt_attack(pokemon: dict):
    if pokemon['hp'] <= 0:
        raise PokemonFaintedError(
            f"{pokemon['name']} has fainted and cannot battle!"
        )

pikachu = {"name": "Pikachu", "hp": 0}

# Catch the specific type
try:
    attempt_attack(pikachu)
except PokemonFaintedError as e:
    print(f"Specific catch: {e}")

# Catch via parent — also works!
try:
    attempt_attack(pikachu)
except BattleError as e:
    print(f"Category catch (BattleError): {e}")

try:
    attempt_attack(pikachu)
except PokemonError as e:
    print(f"Root catch (PokemonError): {e}")

---

## **Adding a __str__ Method**

Override `__str__` to control exactly how the exception prints when it appears in a traceback or when you call `str(e)`. This is useful for rich, multi-line error messages.

In [ ]:
class BattleError(Exception):
    """Detailed battle error with attacker, defender, and turn context."""

    def __init__(self, message: str, attacker: str, defender: str, turn: int):
        super().__init__(message)
        self.attacker = attacker
        self.defender = defender
        self.turn     = turn

    def __str__(self):
        return (
            f"BattleError on turn {self.turn}:\n"
            f"  Attacker : {self.attacker}\n"
            f"  Defender : {self.defender}\n"
            f"  Problem  : {self.args[0]}"
        )

def process_turn(attacker, defender, turn):
    if attacker['hp'] <= 0:
        raise BattleError(
            "Attacker has already fainted",
            attacker['name'],
            defender['name'],
            turn
        )

fainted = {"name": "Pikachu",   "hp": 0}
onix    = {"name": "Onix",      "hp": 70}

try:
    process_turn(fainted, onix, turn=3)
except BattleError as e:
    print(e)   # Uses our __str__ method

---

## **Practical: Complete Pokemon Exception Module**

In [ ]:
# --- pokemon_exceptions.py (inline for the notebook) ---

class PokemonError(Exception):
    """Root exception for all Pokemon game errors."""

class InvalidLevelError(PokemonError):
    def __init__(self, name, level):
        self.name, self.level = name, level
        super().__init__(f"{name}: level {level} must be 1–100")

class TeamFullError(PokemonError):
    def __init__(self, pokemon_name, team_size=6):
        super().__init__(
            f"Cannot add {pokemon_name}: team already has {team_size} members"
        )

class PokemonFaintedError(PokemonError):
    def __init__(self, name):
        self.name = name
        super().__init__(f"{name} has fainted and cannot battle")

class PokemonNotFoundError(PokemonError):
    def __init__(self, name):
        self.name = name
        super().__init__(f"'{name}' is not registered in this Pokedex")

# --- game code using the exceptions ---

POKEDEX = {"Pikachu": "Electric", "Charizard": "Fire", "Blastoise": "Water"}

def add_to_team(team, name, level):
    if name not in POKEDEX:
        raise PokemonNotFoundError(name)
    if len(team) >= 6:
        raise TeamFullError(name)
    if not (1 <= level <= 100):
        raise InvalidLevelError(name, level)
    team.append({"name": name, "level": level})

def use_in_battle(pokemon):
    if pokemon.get("hp", 1) <= 0:
        raise PokemonFaintedError(pokemon["name"])
    return f"{pokemon['name']} attacks!"

# Try the system
team = []
tests = [
    ("Pikachu",  25),
    ("Charizard", 36),
    ("Eevee",    10),   # Not in Pokedex
    ("Blastoise", 200), # Bad level
]

for name, level in tests:
    try:
        add_to_team(team, name, level)
        print(f"  ✓ Added {name}")
    except PokemonNotFoundError as e:
        print(f"  ✗ Not found: {e}")
    except InvalidLevelError as e:
        print(f"  ✗ Bad level: {e}")
    except PokemonError as e:
        print(f"  ✗ Game error: {e}")

print(f"\nTeam: {[p['name'] for p in team]}")

---

## **Practice Exercises**

### **Task 1: Simple Custom Exception**

Create `InvalidMoveError` and raise it with a message.

**Expected Output:**
```
InvalidMoveError: 'HyperBeam' is not a valid move for Pikachu
```

In [ ]:
# Your code here:


### **Task 2: Exception with Attributes**

Create `CatchFailedError` that stores `pokemon_name` and `roll` as attributes.

**Expected Output:**
```
Pokemon: Mewtwo
Roll: 0.95
```

In [ ]:
# Your code here:


### **Task 3: Hierarchy**

Create `PokemonError` → `BattleError` → `OutOfPPError`. Show that catching `BattleError` catches `OutOfPPError`.

**Expected Output:**
```
BattleError caught: Thunderbolt has 0 PP remaining
```

In [ ]:
# Your code here:


### **Task 4: Custom __str__**

Override `__str__` to format the error message with the Pokemon name on its own line.

**Expected Output:**
```
PokemonFaintedError:
  Pokemon : Pikachu
  Reason  : HP reached 0
```

In [ ]:
# Your code here:


### **Task 5: Catch via Parent**

Raise `TeamFullError` (subclass of `PokemonError`) and catch it via the parent `PokemonError`.

**Expected Output:**
```
PokemonError caught: team is full
```

In [ ]:
# Your code here:


### **Task 6: Don't Catch Sibling**

Show that catching `BattleError` does NOT catch `TeamError` (they're siblings).

**Expected Output:**
```
TeamError escaped BattleError handler
```

In [ ]:
class PokemonError(Exception): pass
class BattleError(PokemonError): pass
class TeamError(PokemonError): pass

# Your code here:


### **Task 7: Document Your Exception**

Create `SaveFileCorruptedError` with a docstring, attributes for `filename` and `reason`, and a useful `__str__`.

**Expected Output:**
```
Save file 'ash.json' is corrupted: invalid JSON
```

In [ ]:
# Your code here:


### **Task 8: Multi-level Hierarchy**

Build `PokemonError` → `ItemError` → `ItemNotFoundError` and `ItemDepletedError`. Catch both via `ItemError`.

**Expected Output:**
```
ItemError: Potion not found in bag
ItemError: Max Potion supply exhausted
```

In [ ]:
# Your code here:


### **Task 9: Use in Real Code**

Write `catch_pokemon(name, catch_rate)` that raises `CatchFailedError` or `PokemonNotFoundError` as appropriate.

**Expected Output:**
```
Pikachu caught!   (or CatchFailedError)
PokemonNotFoundError: 'Eevee' not registered
```

In [ ]:
# Your code here:


### **Task 10: Full Exception Module**

Write the hierarchy `PokemonError` → `BattleError`, `TeamError`, `PokedexError` with one concrete subclass each. Demonstrate raising and catching each one.

**Expected Output:**
```
✓ BattleError caught
✓ TeamError caught
✓ PokedexError caught
```

In [ ]:
# Your code here:


---

## **Summary**

- Create custom exceptions by inheriting from `Exception` (or any subclass)
- Even an empty class body gives you a new, uniquely catchable type
- Override `__init__` to store extra attributes alongside the message
- Call `super().__init__(message)` to set the error message properly
- Override `__str__` to control the printed representation
- Build hierarchies: `PokemonError` → `BattleError` → `PokemonFaintedError`
- Catching a parent catches all its descendants
- Catching a sibling does NOT catch a different branch
- Name exceptions as nouns ending in `Error`: `InvalidLevelError`, `TeamFullError`

---

## **Quick Reference**

```python
# Minimal custom exception
class MyError(Exception):
    """Docstring."""

# With attributes
class InvalidLevelError(Exception):
    def __init__(self, name, level):
        self.name  = name
        self.level = level
        super().__init__(f"{name}: bad level {level}")

# Hierarchy
class PokemonError(Exception): pass
class BattleError(PokemonError): pass
class PokemonFaintedError(BattleError): pass

# Raising
raise InvalidLevelError("Pikachu", 150)

# Catching — any of these works
except PokemonFaintedError: ...  # Most specific
except BattleError: ...          # Catches all battle errors
except PokemonError: ...         # Catches all game errors
```